In [1]:
import pandas as pd
from tqdm import tqdm

In [10]:
# We load our Excel sheets to pandas DataFrames
df_drug1 = pd.read_excel('drug1.xlsx', engine='openpyxl')
df_labsum = pd.read_csv('labsum.csv')
df_outcome = pd.read_csv('outcome.csv')

In [11]:
# Just to be sure we don't modify anything, we take a copy of said DataFrames
df_drug1_test = df_drug1.copy(deep=True)
df_labsum_test = df_labsum.copy(deep=True)
df_outcome_test = df_outcome.copy(deep=True)

In [12]:
# We create out dataframe to fill
df_merged = pd.DataFrame(index=sorted(set(df_outcome_test['CASEID'].to_list())))

In [13]:
for row in range(df_drug1_test.shape[0]):
    df_drug1_test.at[row, 'ID'] = str(df_drug1_test.at[row, 'CASEID']) + str(df_drug1_test.at[row, 'VISIT'])

    
df_drug1_visits = df_drug1_test.copy()
df_drug1_visits = df_drug1_visits.drop_duplicates(subset='ID', keep='first', ignore_index=True)


for row in tqdm(df_merged.index.to_list()):
    df_merged.at[row, 'sum_visits'] =  int(df_drug1_visits[(df_drug1_visits['CASEID'] == row)].shape[0])
    df_merged.at[row, 'first_visit'] = df_drug1_visits[(df_drug1_visits['CASEID'] == row)]['DCMPD1'].min()
    df_merged.at[row, 'chg_treatment'] = (df_drug1_visits[(df_drug1_visits['CASEID'] == row)].NMEDD1 == 1).sum()    
    df_merged.at[row, 'set_drugs'] = len(set(df_drug1_visits[(df_drug1_visits['CASEID'] == row)]
                                                     ['DRUGD1'].to_list())) + 1
    
    list_drugs = df_drug1_visits[(df_drug1_visits['CASEID'] == row)]['DRUGD1'].to_list()
    df_drug_years = df_drug1_visits[(df_drug1_visits['CASEID'] == row)][['DRUGD1', 'DCMPD1']].sort_values(by='DCMPD1')
    df_drug_years = df_drug_years.drop_duplicates(subset='DRUGD1', keep='first')
    df_drug_years = df_drug_years.set_index(keys='DRUGD1')
    
    for drug in set(list_drugs):
        df_merged.at[row, '{}{}{}'.format('drug_', drug, '_amount')] = list_drugs.count(drug)
        df_merged.at[row, '{}{}{}'.format('drug_', drug, '_years')] = df_drug_years.at[drug, 'DCMPD1']

    df_outcome_caseid = df_outcome_test[(df_outcome_test['CASEID'] == row)].reset_index(drop=True)
    
    try:
        df_merged.at[row, 'death_year'] = int(df_outcome_caseid[df_outcome_caseid['OUTCDTH'] == 1].DATEREPT)
        df_merged.at[row, 'OUTCAIDS_ones'] = df_outcome_caseid[df_outcome_caseid['OUTCAIDS'] == 1].shape[0]
        
    except Exception as e:
        df_merged.at[row, 'death_year'] = None
        df_merged.at[row, 'OUTCAIDS_ones'] = None
    
    change_detector = 0
    change_counter = 0
    
    for i in range(df_outcome_caseid.shape[0]):
        
        if (df_outcome_caseid.at[i, 'OUTCAIDS'] == 0) and (change_detector == 1):
            change_detector = df_outcome_caseid.at[i, 'OUTCAIDS']
            change_counter += 1        
            
        elif (df_outcome_caseid.at[i, 'OUTCAIDS'] == 1) and (change_detector == 0):
            change_detector = 1
        
        else:
            pass
            
    df_merged.at[row, 'OUTCAIDS_change'] = change_counter
    
    
df_merged['alive_dead'] = df_outcome_test['OUTCDTH']
df_merged['death_AIDS'] = df_outcome_test['DTHCLASS']
df_merged['death_cause'] = df_outcome_test['SPECCODE']

    
cols = list(df_merged)
cols.insert(0, cols.pop(cols.index('sum_visits')))
cols.insert(1, cols.pop(cols.index('first_visit')))
cols.insert(2, cols.pop(cols.index('set_drugs')))
cols.insert(3, cols.pop(cols.index('chg_treatment')))
cols.insert(4, cols.pop(cols.index('alive_dead')))
cols.insert(5, cols.pop(cols.index('death_cause')))
cols.insert(6, cols.pop(cols.index('death_year')))
cols.insert(7, cols.pop(cols.index('OUTCAIDS_ones')))
cols.insert(8, cols.pop(cols.index('OUTCAIDS_change')))

df_merged = df_merged.loc[:, cols]

display(df_merged)


100%|██████████| 2933/2933 [00:18<00:00, 160.70it/s]


,sum_visits,first_visit,set_drugs,chg_treatment,alive_dead,death_cause,death_year,OUTCAIDS_ones,OUTCAIDS_change,drug_293_amount,...,drug_295_years,drug_238_amount,drug_238_years,drug_193_amount,drug_193_years,drug_-8_amount,drug_-8_years,drug_268_amount,drug_268_years,death_AIDS
1000,3.0,2014.0,3.0,1.0,0,-1.0,NaN,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1002,0.0,NaN,1.0,0.0,0,-1.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1003,10.0,1999.0,5.0,3.0,0,-1.0,2006.0,6.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1005,0.0,NaN,1.0,0.0,0,-1.0,1997.0,5.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1006,34.0,1999.0,7.0,1.0,0,-1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,17.0,1999.0,7.0,1.0,0,-1.0,2007.0,2.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
9993,28.0,2000.0,6.0,3.0,0,-1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
9994,0.0,NaN,1.0,0.0,0,-1.0,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
9995,0.0,NaN,1.0,0.0,0,-1.0,1997.0,3.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


In [24]:
df_merged_test = df_merged.copy(deep=True)
first_cols = [col for col in df_merged_test][:9]
last_cols = [col for col in df_merged_test][9:]

df_labsum_test = df_labsum.copy(deep=True)
labsum_index = df_labsum.index.tolist()
labsum_cols = [f'labsum_{col}' for col in df_labsum_test]

for index in tqdm(df_merged_test.index):
    if index in labsum_index:
        for col in df_labsum_test:
            df_merged_test.at[index, f'labsum_{col}'] = df_labsum_test.at[index, col]
    
df_merged_test = df_merged_test[first_cols + labsum_cols + last_cols]
display(df_merged_test)

100%|██████████| 2933/2933 [00:00<00:00, 4766.15it/s]


,sum_visits,first_visit,set_drugs,chg_treatment,alive_dead,death_cause,death_year,OUTCAIDS_ones,OUTCAIDS_change,labsum_CASEID,...,drug_295_years,drug_238_amount,drug_238_years,drug_193_amount,drug_193_years,drug_-8_amount,drug_-8_years,drug_268_amount,drug_268_years,death_AIDS
1000,3.0,2014.0,3.0,1.0,0,-1.0,NaN,NaN,0.0,1125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1002,0.0,NaN,1.0,0.0,0,-1.0,NaN,NaN,0.0,1125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1003,10.0,1999.0,5.0,3.0,0,-1.0,2006.0,6.0,2.0,1125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1005,0.0,NaN,1.0,0.0,0,-1.0,1997.0,5.0,1.0,1125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
1006,34.0,1999.0,7.0,1.0,0,-1.0,NaN,NaN,1.0,1125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9988,17.0,1999.0,7.0,1.0,0,-1.0,2007.0,2.0,2.0,2320.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
9993,28.0,2000.0,6.0,3.0,0,-1.0,NaN,NaN,1.0,2320.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
9994,0.0,NaN,1.0,0.0,0,-1.0,NaN,NaN,0.0,2320.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1
9995,0.0,NaN,1.0,0.0,0,-1.0,1997.0,3.0,1.0,2320.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1


In [25]:
df_merged_test.to_excel('20210327_merged_files.xlsx')